In [1]:
from TopoInteraction.TI_Loss import TI_Loss, DiceLoss
from torch.nn import CrossEntropyLoss
import torch
import os 

In [2]:
print(torch.__version__)

2.2.0+cu121


In [2]:
from model import Click_ref

In [3]:
from train import train

NameError: name 'run' is not defined

In [7]:
import sys
# from pathlib import Path
# sys.path.append(str(Path(__file__).resolve().parent.parent))
from config import *
from model import Click_ref
import neptune
# from tiloss import Finalloss
from dataloader import dataloaders
import torch
from TopoInteraction.TI_Loss import final_loss
# torch.cuda.empty_cache()

click_ref = Click_ref(3, 3)
# click_ref.load_state_dict(torch.load(path_weights_click_ref))
optimizer = torch.optim.Adam(click_ref.parameters(), lr=3e-3)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
run = neptune.init_run(
    project="aureliensihab/deep-icy",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwZjdkOTI0Yy1iOGJkLTQyMzEtYmEyOC05MmFmYmFhMWExNTMifQ==",
)
run["config/Hyperparameters"] = parameters
run["config/optimizer"] = "Adam"

train(click_ref, optimizer, dataloaders["train"], dataloaders["val"], epochs=500)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/aureliensihab/deep-icy/e/DEEP-393


KeyboardInterrupt: 

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

1461548

In [5]:
ti_loss = TI_Loss(dim=2, connectivity=8, inclusion=[[2, 1]], exclusion=[])
ce1 = CrossEntropyLoss(reduce=None)
ce2 = CrossEntropyLoss(reduce=None)
d1 = DiceLoss()
d2 = DiceLoss()
lbd_dice = 1e-2
dice_weight = 1., ti_weight = 1e-4

def final_loss(y_true, y_pred, stardist, click_map):
    return (
        torch.mean(
            torch.sum(
                click_map * (ce1(y_pred, y_true) +  d1(y_pred, y_true)),
                dim=(1, 2, 3),
            )
        )
        + 1.
        * torch.mean(
            torch.sum(
                (1 - click_map)
                * (ce2(y_pred, stardist) +  d2(y_pred, stardist)),
                dim=(1, 2, 3),
            )
        )
        + 1e-4 * ti_loss(y_pred, y_true)
    )

In [ ]:
one_hot(